In [16]:
import os
os.makedirs("OUTPUT/", exist_ok=True)

In [17]:
import urllib.request

GITHUB_Y_NPY_URL = "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/y.npy"
urllib.request.urlretrieve(GITHUB_Y_NPY_URL, "OUTPUT/y.npy")

print("✅ y.npy file downloaded successfully!")

✅ y.npy file downloaded successfully!


In [18]:
GITHUB_FILES = {
    "y.npy": "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/y.npy",
    "X_embeddings.npy": "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/X_embeddings.npy",
    "label_encoder.pkl": "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/label_encoder.pkl",
    "logistic_regression_bert.pkl": "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/logistic_regression_bert.pkl",
    "neural_network_bert.h5": "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/neural_network_bert.h5"
}

for filename, url in GITHUB_FILES.items():
    if not os.path.exists(f"OUTPUT/{filename}"):
        urllib.request.urlretrieve(url, f"OUTPUT/{filename}")
        print(f"✅ {filename} downloaded successfully!")
    else:
        print(f"✔️ {filename} already exists.")

✔️ y.npy already exists.
✔️ X_embeddings.npy already exists.
✔️ label_encoder.pkl already exists.
✔️ logistic_regression_bert.pkl already exists.
✔️ neural_network_bert.h5 already exists.


In [19]:
!pip install reportlab

import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from tensorflow.keras.models import load_model

# 1. Load Necessary Files**
print("Loading saved models and data...")

# Load labels
y_test = np.load("OUTPUT/y.npy")
label_encoder = joblib.load("OUTPUT/label_encoder.pkl")
X_test_bert = np.load("OUTPUT/X_embeddings.npy")  # BERT-based embeddings

# Load trained models
logreg_model = joblib.load("OUTPUT/logistic_regression_bert.pkl")

nn_model = load_model("OUTPUT/neural_network_bert.h5")

print("Models and data loaded successfully!")

# 2. Generate Predictions**
print(" Generating predictions...")

logreg_preds = logreg_model.predict(X_test_bert)
nn_preds = np.argmax(nn_model.predict(X_test_bert), axis=1)

#3. Classification Reports**
logreg_report = classification_report(y_test, logreg_preds, target_names=label_encoder.classes_)
nn_report = classification_report(y_test, nn_preds, target_names=label_encoder.classes_)

print("\n Classification Report - Logistic Regression\n", logreg_report)
print("\n Classification Report - Neural Network\n", nn_report)

# 4. Save Classification Report as PDF**
def save_combined_report_as_pdf(filename="OUTPUT/genre_classification_report.pdf"):
    c = canvas.Canvas(filename, pagesize=letter)
    textobject = c.beginText(50, 750)

    textobject.textLine(" Genre Classification Report")
    textobject.textLine("──────────────────────────────────")
    textobject.textLine(" Logistic Regression Results:")
    for line in logreg_report.split("\n"):
        textobject.textLine(line)

    textobject.textLine("\n Neural Network Results:")
    for line in nn_report.split("\n"):
        textobject.textLine(line)

    c.drawText(textobject)
    c.save()
    print(" Genre classification report saved to OUTPUT/genre_classification_report.pdf!")

save_combined_report_as_pdf()

# 5. Generate and Save Confusion Matrices**
def save_confusion_matrices():
    print(" Creating confusion matrices...")

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Logistic Regression Confusion Matrix
    sns.heatmap(confusion_matrix(y_test, logreg_preds), annot=True, fmt="d", cmap="Blues",
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, ax=axes[0])
    axes[0].set_title("Confusion Matrix - Logistic Regression")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("Actual")

    # Neural Network Confusion Matrix
    sns.heatmap(confusion_matrix(y_test, nn_preds), annot=True, fmt="d", cmap="Blues",
                xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_, ax=axes[1])
    axes[1].set_title("Confusion Matrix - Neural Network")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("Actual")

    plt.tight_layout()
    plt.savefig("OUTPUT/confusion_matrix.png")
    plt.close()
    print(" Confusion matrices saved to OUTPUT/confusion_matrix.png!")

save_confusion_matrices()

print("\n Analysis Complete!** All outputs saved in the OUTPUT/ directory.")


Loading saved models and data...


Models and data loaded successfully!
 Generating predictions...
887/887 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step

 Classification Report - Logistic Regression
               precision    recall  f1-score   support

       blues       0.39      0.32      0.35      4604
     country       0.49      0.59      0.54      5445
     hip hop       0.76      0.61      0.67       904
        jazz       0.41      0.35      0.38      3845
         pop       0.40      0.52      0.45      7042
      reggae       0.50      0.40      0.44      2498
        rock       0.46      0.35      0.40      4034

    accuracy                           0.45     28372
   macro avg       0.49      0.45      0.46     28372
weighted avg       0.45      0.45      0.44     28372


 Classification Report - Neural Network
               precision    recall  f1-score   support

       blues       0.45      0.07      0.12      4604
     country       0.44      0.55      0.49      5445
     hip hop       0.98      0.42      0.59   